In [1]:
import cvxpy as cp
import numpy as np

In [2]:
cost = np.array([3,3]).reshape(-1,1)
demand = np.array([[1.5,1.5],[2,2],[3.5,3.5],[4,4],[5.5,5.5]]).reshape(-1,2)
produce = np.array([8,8]).reshape(-1,1)

In [3]:
def solve_second_stage_subproblem(produce,all_demand):

    selling_price = np.array([5,5]).reshape(-1,1)
    salvage_value = np.array([1,1]).reshape(-1,1)
    size = 2
    duals = []
    vs = []

    for demand in all_demand:

        shape = (size,1)

        sell = cp.Variable(shape=(size,1))
        salvage = cp.Variable(shape=(size,1))

        objective = cp.Minimize(-1*(selling_price.T@sell)-1*(salvage_value.T@salvage))
        constraints =  [sell+salvage<=produce,sell<=demand[0].reshape(-1,1),salvage>=np.zeros_like(shape).reshape(-1,1),sell>=np.zeros_like(shape).reshape(-1,1)]

        problem = cp.Problem(objective, constraints)
        problem.solve()

        dual_value = problem.constraints[0].dual_value

        duals.append(dual_value)
        vs.append(problem.value)
    
    return np.array(duals).reshape(-1,size),np.array(vs).reshape(-1,1)

In [4]:
duals,qs = solve_second_stage_subproblem(produce,demand)

In [5]:
def solve_first_stage_problem(produce,limit,g_ks,alpha_ks):
    
    size = 2
    cost = np.array([3,3]).reshape(-1,1)

    produce  = cp.Variable(shape=(2,1))
    v = cp.Variable(shape=(1,1))

    constraints = []
    for i in range(0,len(g_ks)):
        constraints.append(np.array(g_ks[i]).T@produce+np.array(alpha_ks[i])<=v)

    constraints.extend([produce>=0,v>=-1000000,produce>=np.full((size,1),limit),produce<=np.full((size,1),100)])

    objective = cp.Minimize((cost.T@produce)+v)

    problem = cp.Problem(objective, constraints)
    problem.solve()

    return produce.value, v.value

In [6]:
produce = np.array([3,3]).reshape(-1,1)
limit = -100000

g_ks = []
alpha_ks = []

for i in range(0,100):
    duals,qs = solve_second_stage_subproblem(produce,demand)

    senario = 5
    senario_prob = (1/senario)*(np.ones_like(demand[:,0])).reshape(-1,1)
    g_ks_temp = (-senario_prob.T@duals).T
    alpha_ks_temp = senario_prob.T@qs - g_ks_temp.T@produce

    

    g_ks.append(g_ks_temp)
    alpha_ks.append(alpha_ks_temp)

    produce,limit= solve_first_stage_problem(produce,limit,g_ks,alpha_ks)
    print(produce)
    # print(limit)

    


[[99.99999996]
 [99.99999996]]
[[4.33333332]
 [4.33333332]]
[[3.74999999]
 [3.74999999]]
[[3.49999999]
 [3.49999999]]
[[1.26296402]
 [5.73703595]]
[[3.1040427 ]
 [3.89595727]]
[[3.5       ]
 [3.49999998]]
[[3.5000003 ]
 [3.49999967]]
[[3.50000001]
 [3.50000006]]
[[3.49999997]
 [3.50000001]]
[[3.49999998]
 [3.5       ]]
[[3.49999998]
 [3.5       ]]
[[3.49999997]
 [3.5       ]]
[[3.49999998]
 [3.5       ]]
[[3.49999999]
 [3.50000007]]
[[3.49999999]
 [3.49999995]]
[[3.49999998]
 [3.5       ]]
[[3.5]
 [3.5]]
[[3.49999998]
 [3.49999999]]
[[3.49999999]
 [3.5       ]]
[[3.50000007]
 [3.50000003]]
[[3.50000001]
 [3.49999999]]
[[3.5]
 [3.5]]
[[3.49999999]
 [3.5       ]]
[[3.49999999]
 [3.49999999]]
[[3.49999999]
 [3.49999999]]
[[3.49999998]
 [3.49999999]]
[[3.49999998]
 [3.49999999]]
[[3.49999997]
 [3.50000001]]
[[3.49999998]
 [3.49999999]]
[[3.50000003]
 [3.49999996]]
[[3.49999988]
 [3.50000006]]
[[3.50000004]
 [3.49999999]]
[[3.50000006]
 [3.49999998]]
[[3.50000004]
 [3.49999998]]
[[3.5      